# DevSavvy: Run an Ollama Server on Google Colab & Expose via ngrok 🚀

## Step 1: Install Ollama ✅
### Ollama provides a simple API for managing LLMs.

In [ ]:
!curl -fsSL https://ollama.ai/install.sh | sh

## Step 2: Install & Setup ngrok ✅
### Download and extract the latest ngrok binary

In [9]:
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok-v3-stable-linux-amd64.tgz ngrok

## Authenticate ngrok

* a ngrok auth token must be aquired in order to use it.

* Follow the instruction below:
https://ngrok.com/docs/getting-started/

* Then replace your AuthToken with <NGROK_AUTH_TOKEN> below:


In [ ]:
from google.colab import userdata

# Ambil token dari secrets
NGROK_AUTH_TOKEN = userdata.get('ngrok')

# Pakai token ini untuk setup ngrok
!./ngrok authtoken {NGROK_AUTH_TOKEN}



## Step 3: Run Ollama & Expose API via ngrok 🚀
### Start the Ollama server in the background

###Start Ollama Serve

In [ ]:
import subprocess

# Simpan ke variabel agar bisa dihentikan nanti
ollama_proc = subprocess.Popen(["ollama", "serve"])
print(f"🟢 ollama serve dijalankan | PID: {ollama_proc.pid}")


###Start ngrok Serve

In [ ]:
import subprocess
import time
import requests

# 1. Jalankan ngrok secara background
ngrok_proc = subprocess.Popen([
    "./ngrok", "http", "11434", "--host-header=localhost:11434"
])
print(f"🟢 ngrok dijalankan | PID: {ngrok_proc.pid}")

# 2. Tunggu ngrok aktif
time.sleep(5)  # penting agar ngrok siap dulu

# 3. Ambil link dari ngrok API lokal
try:
    res = requests.get("http://localhost:4040/api/tunnels")
    tunnels = res.json()['tunnels']
    public_url = tunnels[0]['public_url']
    print(f"🔗 Link publik ngrok: {public_url}")
except Exception as e:
    print("❌ Gagal mengambil URL ngrok:", e)


###Start llama3.2

In [ ]:
import subprocess

# Jalankan model LLM lokal melalui Ollama
llama_proc = subprocess.Popen(["ollama", "run", "llama3.2"])
print(f"🟢 Model llama3.2 dijalankan | PID: {llama_proc.pid}")


###chek serve

In [ ]:
import subprocess

print("📋 Daftar proses terkait yang sedang berjalan:\n")

# Proses yang sedang aktif untuk 'ollama', 'ngrok', dan 'llama3'
for keyword in ["ollama", "ngrok", "llama3"]:
    print(f"\n🔍 Proses aktif: {keyword}")
    result = subprocess.run(["ps", "-ef"], capture_output=True, text=True)
    lines = result.stdout.splitlines()
    matched = [line for line in lines if keyword in line]
    if matched:
        for line in matched:
            print(line)
    else:
        print(f"❌ Tidak ada proses '{keyword}' ditemukan.")


###Stop serve ollama

In [ ]:
try:
    ollama_proc.terminate()
    ollama_proc.wait()
    print("🛑 Proses 'ollama serve' dihentikan dan dibersihkan.")
except:
    print("⚠️ Gagal menghentikan 'ollama serve' (mungkin belum dijalankan atau sudah mati).")


###Stop serve ngrok

In [ ]:
try:
    ngrok_proc.terminate()
    ngrok_proc.wait()
    print("🛑 ngrok dihentikan dan dibersihkan.")
except:
    print("⚠️ Gagal menghentikan 'ngrok' (mungkin belum dijalankan).")


###Stop serve llama3.2

In [ ]:
try:
    llama_proc.terminate()
    llama_proc.wait()
    print("🛑 llama3.2 dihentikan dan dibersihkan.")
except:
    print("⚠️ Gagal menghentikan 'ollama run llama3.2' (mungkin belum dijalankan).")


##✅ 1. Install Semua Dependensi (WAJIB)

In [ ]:
# Wajib install semua modul
!pip install -U llama-index langchain langchain-community sentence-transformers

##✅ 2. Load Dokumen dari Google Drive (LlamaIndex)

In [ ]:
from llama_index.core import SimpleDirectoryReader

# Arahkan ke folder dokumen distrik
folder_path = "/content/drive/MyDrive/Distrik"

# Load semua file termasuk dari subfolder
documents = SimpleDirectoryReader(input_dir=folder_path, recursive=True).load_data()

# Tampilkan jumlah & nama file
print(f"\n✅ Total dokumen terbaca: {len(documents)}\n")

for i, doc in enumerate(documents, start=1):
    file_path = doc.metadata.get("file_path", "Tidak diketahui")
    print(f"{i}. {file_path}")


In [ ]:
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext

# Inisialisasi embedding model lokal
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Buat konteks layanan dengan embedding lokal
service_context = ServiceContext.from_defaults(embed_model=embed_model)

# Bangun index dari dokumen yang sudah diload sebelumnya
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

print("✅ Index selesai dibuat.")
